In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
leis = pd.read_json('leis.json')
leis.drop(['documento'], inplace=True, axis=1)
print(leis.info())
print(leis.nunique())
leis

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6033 entries, 0 to 6032
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   titulo     6033 non-null   object
 1   categoria  6033 non-null   object
 2   resumo     6033 non-null   object
 3   texto      6033 non-null   object
dtypes: object(4)
memory usage: 188.7+ KB
None
titulo       6033
categoria       8
resumo       4961
texto        6029
dtype: int64


,titulo,categoria,resumo,texto
0,"DECRETO Nº 8854, de 28 de fevereiro de 2013.",Decretos,DELEGA COMPETÊNCIA À SECRETARIA MUNICIPAL DE P...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
1,"DECRETO Nº 8849, de 25 de fevereiro de 2013.",Decretos,ABRE CRÉDITO SUPLEMENTAR AO ORÇAMENTO DO MUNIC...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
2,"DECRETO Nº 8853, de 27 de fevereiro de 2013.",Decretos,NOMEIA MEMBROS DO CONSELHO MUNICIPAL DE DESENV...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
3,"DECRETO Nº 8967, de 17 de julho de 2013",Decretos,ALTERA O QUADRO DE DETALHAMENTO DE DESPESA DO ...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
4,"DECRETO Nº 8982, de 30 de julho de 2013",Decretos,AUTORIZA O FUNCIONAMENTO DE ESTABELECIMENTOS C...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
...,...,...,...,...
6028,RESOLUÇÃO Nº 125/1980,Resoluções,DISPÕE SOBRE A CONCESSÃO DE TÍTULO DE CIDADÃO ...,Faço saber que a Câmara Municipal aprovou e eu...
6029,RESOLUÇÃO Nº 403/2003,Resoluções,AUTORIZA A MESA DIRETIVA DO PODER LEGISLATIVO ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6030,RESOLUÇÃO Nº 492/2014,Resoluções,INSTITUI A SEGUNDA SEMANA DO MÊS DE AGOSTO EM ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6031,RESOLUÇÃO Nº 382/2001,Resoluções,CRIA A MEDALHA VEREADOR DIVAL FIGUEIREDO MACHA...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."


In [3]:
leis.iloc[len(leis)-1, 3]

'A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, na conformidade do artigo 70, Inciso V, da Lei Municipal nº37, de 05 de Abril de 1990 e, artigos 287, § 2º e, 420, do Regimento Interno, promulga a seguinte Resolução:\n\nArt. 1ºDê-se aos dispositivos abaixo mencionados, da Resolução nº393/2002 - Regimento Interno, as seguintes redações:\n\n"Art. 7º A Mesa Diretora da Câmara compor-se-á do Presidente, Primeiro e Segundo Secretários, com mandato de 02 ( dois ) anos, admitida a recondução para a eleição subsequente.\n\n§ 4º Se, hora regimental, não estiver presente o Presidente, abrirá os trabalhos o Vice-Presidente ou, na falta deste, o Primeiro ou Segundo Secretários, na sequência, ou ainda, caso estes não estejam presentes, o Vereador mais votado nas eleições municipais."\n\n"Art. 33 Compete, privativamente, ao Vice-Presidente:"\n\n"Art. 36 ...\n\nI - ...\n\ne) acompanhar e supervisionar a Ata da Sessão, proceder a sua leitura e assiná-la depois do Presidente e do Vice-Presiden

In [4]:
from nltk.corpus import stopwords
import re
import unicodedata

def limpa_texto(text, string=True) -> str or list:
    if (type(text) == float):
        return ''

    # Remove pontuacao, digitos e espacos
    text = ' '.join(re.findall(r'\b[a-zÀ-ú]+\b', text.lower()))
    
    # Remove acentos, cedilhas etc
    nfkd_form = unicodedata.normalize('NFKD', text)
    text = ''.join([char for char in nfkd_form if not unicodedata.combining(char)])

    # Remove stopwords
    my_stopwords = stopwords.words('portuguese')
    my_words = ['feira', 'santana', 'art', 'municipal', 'lei', 'r', 
    'prefeito', 'câmara', 'municipio', 'data', 'seguinte', 'disposições',
    'estado', 'bahia', 'vigor', 'secretário', 'decreto', 'projeto', 
    'iii', 'i', 'ii',  'contrário', 'presidente', 'artigo',
    'faço', 'parágrafo', 'executivo', 'gabinete', 'único', 'sanciono', 
    'desta', 'v', 'iv', 'autoria', 'através', 'deste', 'vice', 'autor',
    'qualquer', 'b', 'decretou', 'execução', 'sobre', 'das', 'decorrentes',
    'decreta', 'resolução', 'geral', 'uso', 'ato', 'diretiva', 'exercício',
    'seguintes', 'meio', 'm', 'c', 'correrão']
    my_stopwords = my_stopwords + my_words

    text = [word for word in text.split() if word not in my_stopwords]

    if string:
        return ' '.join(text)
    else:
        return text

# Semelhança de documentos

In [7]:
leis['texto_limpo'] = leis['texto'].apply(limpa_texto)

In [8]:
# Constrói matriz de documentos
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer # Trasnformar pra TF IDF depois pra testar
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(leis['texto_limpo'])
X

<6033x28875 sparse matrix of type '<class 'numpy.int64'>'
	with 534596 stored elements in Compressed Sparse Row format>

In [9]:
# Quero pegar o primeiro documento e achar qual documento mais similar a ele
# A partir daí, ler as leis pra ver se faz sentido, se são similares etc

# Pra achar os similares, computar a similaridade do cosseno
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

cos_sim = cosine_similarity(X, dense_output=True)
cos_sim_argsort = np.argsort(cos_sim)

(6033, 6033)

In [12]:
# cos_dist guarda uma matriz de similaridades cosseno
# Na primeira linha da matriz, estão as similaridades do primeiro texto pra todos os outros
# Capturar os 3 textos mais similares (tirando ele mesmo) e ler
max_sim_index = cos_sim_argsort[:,-2]

for i, ind in enumerate(max_sim_index):
    max_sim = cos_sim[i, ind]

max_sim_overall = np.max(max_sim)
print(f'Similaridade maxima dos docs: {max_sim_overall}')

orig_text_ind = np.argmax(max_sim)
text_mais_sem_ind = max_sim_index[orig_text_ind]

Similaridade maxima dos docs: 0.5439544481389702
atribuicoes considerando necessidade implementar programa governo federal denominado crack possivel vencer busca aumentar oferta tratamento atencao usuarios aludido elemento toxico considerando julho dispoe criacao conselho antidrogas nao estabelece vinculo conselho secretaria prevencao violencia promocao direitos humanos tampouco outra secretaria considerando enfim competencia inerente secretaria prevencao violencia promocao direitos humanos concomitantemente desenvolvimento politicas publicas modo celere eficiente especial combate drogas ilicitas delegada competencia secretaria prevencao violencia promocao direitos humanos implantacao conselho antidrogas comad respectivas atividades entrara publicacao revogadas disposicoes contrario fevereiro jose ronaldo carvalho mario costa borges chefe carlos antonio moraes lucena procurador mauro oliveira moraes secretario prevencao violencia promocao direitos humanos


- - - TEXTO MAIS SEMELHANTE 

In [37]:
def print_lei_mais_sem(orig_idx : int, sem_idx: int):
    print('- - - LEI COMPARADA {orig_idx}: - - -\n\n')
    print(leis.loc[orig_idx, 'texto'])
    print('\n\n- - - LEI MAIS SEMELHANTE {sem_idx} - - -\n\n')
    print(leis.loc[sem_idx, 'texto'])

print_lei_mais_sem(orig_text_ind, text_mais_sem_ind)

- - - LEI COMPARADA: - - -


atribuicoes considerando necessidade implementar programa governo federal denominado crack possivel vencer busca aumentar oferta tratamento atencao usuarios aludido elemento toxico considerando julho dispoe criacao conselho antidrogas nao estabelece vinculo conselho secretaria prevencao violencia promocao direitos humanos tampouco outra secretaria considerando enfim competencia inerente secretaria prevencao violencia promocao direitos humanos concomitantemente desenvolvimento politicas publicas modo celere eficiente especial combate drogas ilicitas delegada competencia secretaria prevencao violencia promocao direitos humanos implantacao conselho antidrogas comad respectivas atividades entrara publicacao revogadas disposicoes contrario fevereiro jose ronaldo carvalho mario costa borges chefe carlos antonio moraes lucena procurador mauro oliveira moraes secretario prevencao violencia promocao direitos humanos


- - - LEI MAIS SEMELHANTE - - -


poder atribuic

In [19]:
# Calcular similaridade usando TF-IDF agora
# Comparar com a mesma lei original anterior
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X)

cos_sim_tfidf = cosine_similarity(X_tfidf, dense_output=True)
cos_sim_tfidf_idx = np.argsort(cos_sim_tfidf)

Para o texto original, a semelhanca do mais semelhante e: 0.43452539573076604
atribuicoes considerando necessidade implementar programa governo federal denominado crack possivel vencer busca aumentar oferta tratamento atencao usuarios aludido elemento toxico considerando julho dispoe criacao conselho antidrogas nao estabelece vinculo conselho secretaria prevencao violencia promocao direitos humanos tampouco outra secretaria considerando enfim competencia inerente secretaria prevencao violencia promocao direitos humanos concomitantemente desenvolvimento politicas publicas modo celere eficiente especial combate drogas ilicitas delegada competencia secretaria prevencao violencia promocao direitos humanos implantacao conselho antidrogas comad respectivas atividades entrara publicacao revogadas disposicoes contrario fevereiro jose ronaldo carvalho mario costa borges chefe carlos antonio moraes lucena procurador mauro oliveira moraes secretario prevencao violencia promocao direitos humanos



In [20]:
text_sem_ind = cos_sim_tfidf_idx[orig_text_ind, -2]
sem_tfidf = cos_sim_tfidf[orig_text_ind, text_sem_ind]

print(f'Para o texto original, a semelhanca do mais semelhante e: {sem_tfidf}')
print_lei_mais_sem(orig_text_ind, text_sem_ind)

Para o texto original, a semelhanca do mais semelhante e: 0.43452539573076604
atribuicoes considerando necessidade implementar programa governo federal denominado crack possivel vencer busca aumentar oferta tratamento atencao usuarios aludido elemento toxico considerando julho dispoe criacao conselho antidrogas nao estabelece vinculo conselho secretaria prevencao violencia promocao direitos humanos tampouco outra secretaria considerando enfim competencia inerente secretaria prevencao violencia promocao direitos humanos concomitantemente desenvolvimento politicas publicas modo celere eficiente especial combate drogas ilicitas delegada competencia secretaria prevencao violencia promocao direitos humanos implantacao conselho antidrogas comad respectivas atividades entrara publicacao revogadas disposicoes contrario fevereiro jose ronaldo carvalho mario costa borges chefe carlos antonio moraes lucena procurador mauro oliveira moraes secretario prevencao violencia promocao direitos humanos



In [22]:
# Ok, mostraram a mesma lei!
# A semelhanca usando tfidf eh menor
# Será que sempre mostram a mesma lei ou existe alguma dif?
# Se existe diff, qual o melhor metodo? Contando ou tfidf?
max_sim_idx_tfidf = cos_sim_tfidf_idx[:,-2]

idx_iguais = (max_sim_index == max_sim_idx_tfidf)
print(idx_iguais)

[ True False  True ... False  True  True]


In [25]:
# Os semelhantes pra segunda lei e pra anti-penultima sao diferentes
# Vamos dar uma olhada
print('Segunda lei e lei mais semelhante de acordo com count:\n\n')
print_lei_mais_sem(1, max_sim_index[1])

Segunda lei e lei mais semelhante de acordo com count:


atribuicoes legais base autorizacao contida dezembro aberto credito suplementar orcamento valor cento noventa nove mil reais conforme detalhamento abaixo class programatica economica fonte valor inst total tabela formatada equipe tecnica leismunicipais br recursos disponiveis acorrer despesas presente credito suplementar correrao conta anulacoes dotacoes abaixo detalhadas class programatica economica fonte valor inst total tabela formatada equipe tecnica leismunicipais br contabilidade autorizada efetuar registros contabeis necessarios cumprimento entra publicacao fevereiro jose ronaldo carvalho


- - - TEXTO MAIS SEMELHANTE - - -


atribuicoes legais base autorizacao contida dezembro aberto credito suplementar orcamento valor seiscentos cinquenta mil reais conforme detalhamento abaixo class programatica economica fonte valor inst total tabela formatada equipe tecnica leismunicipais br recursos disponiveis acorrer despesas presen

In [24]:
print('Segunda lei e lei mais semelhante de acordo com tfidf:\n\n')
print_lei_mais_sem(1, max_sim_idx_tfidf[1])

Segunda lei e lei mais semelhante de acordo com tfidf:
atribuicoes legais base autorizacao contida dezembro aberto credito suplementar orcamento valor cento noventa nove mil reais conforme detalhamento abaixo class programatica economica fonte valor inst total tabela formatada equipe tecnica leismunicipais br recursos disponiveis acorrer despesas presente credito suplementar correrao conta anulacoes dotacoes abaixo detalhadas class programatica economica fonte valor inst total tabela formatada equipe tecnica leismunicipais br contabilidade autorizada efetuar registros contabeis necessarios cumprimento entra publicacao fevereiro jose ronaldo carvalho


- - - TEXTO MAIS SEMELHANTE - - -


atribuicoes legais base autorizacao contida dezembro aberto credito suplementar orcamento valor mil reais conforme detalhamento abaixo class programatica economica fonte valor inst total tabela formatada equipe tecnica leismunicipais br recursos disponiveis acorrer despesas presente credito suplementar 

- - - -

Ok, sao bastante similares ainda assim.

Acho que Count se saiu um pouco melhor porque o valor da segunda lei eh mais parecido com a primeira e ambas sao publicacao fevereiro, enquanto a tfidf eh publicacao abril.

Vamos olhar a antepenultima lei:

In [28]:
print('Antpe lei e lei mais semelhante de acordo com count:\n\n')
orig_idx = len(leis) - 3
print_lei_mais_sem(orig_idx, max_sim_index[-3])

Antpe lei e lei mais semelhante de acordo com count:


camara conformidade inciso abril artigos regimento interno resolucao edil gerusa maria bastos s sampaio promulga resolucao instituida ambito camara semana comemoracao dia pais sera segunda semana mes agosto cada ano paragrafo unico efeito exposto caput casa realizara atividades comemorativas alusivas periodo dedicado semana comemoracao dia pais externando direito cada parlamentar homenagear pai sendo figura paternal genitor pessoa despesas resolucao correrao conta verba existente secretaria casa resolucao entrara publicacao revogadas disposicoes contrario mesa camara agosto ver justiniano oliveira franca ver reinaldo miranda vieira filho ver edvaldo lima santos ver roque pereira carmo secretario ver jose carneiro rocha secretario ver welligton andrade jesus secretario


- - - TEXTO MAIS SEMELHANTE - - -


camara conformidade inciso abril artigos regimento interno resolucao mesa resolucao redacao outubro fica considerada dia agente 

In [29]:
print('Antpe lei e lei mais semelhante de acordo com TFIDF:\n\n')
print_lei_mais_sem(orig_idx, max_sim_idx_tfidf[-3])

Antpe lei e lei mais semelhante de acordo com TFIDF:


camara conformidade inciso abril artigos regimento interno resolucao edil gerusa maria bastos s sampaio promulga resolucao instituida ambito camara semana comemoracao dia pais sera segunda semana mes agosto cada ano paragrafo unico efeito exposto caput casa realizara atividades comemorativas alusivas periodo dedicado semana comemoracao dia pais externando direito cada parlamentar homenagear pai sendo figura paternal genitor pessoa despesas resolucao correrao conta verba existente secretaria casa resolucao entrara publicacao revogadas disposicoes contrario mesa camara agosto ver justiniano oliveira franca ver reinaldo miranda vieira filho ver edvaldo lima santos ver roque pereira carmo secretario ver jose carneiro rocha secretario ver welligton andrade jesus secretario


- - - TEXTO MAIS SEMELHANTE - - -


camara conformidade inciso abril artigos regimento interno resolucao edil eremita mota araujo promulga resolucao instituida ambi

Ok, nessa o TF-IDF se sai bem melhor.

A original trata de semana de dia dos pais.

A que o count pegou trata de dia do agente comunitario e cita coisas como saude, endemia, despesas, que não ficaram muit claras pra mim o que são.

Já o TF-IDF trata de semana da familia. Não só familia é parecido com pai, mas tb é uma semnaa. E o texto da lei tb é sobre uma semana comemorativa. 

Vamos ver se existem mais diferenças e quais são estas diferenças:

In [31]:
difs = [i for i, igual in enumerate(idx_iguais) if not igual]
print(difs)
print(len(difs))

[1, 5, 10, 11, 20, 24, 25, 35, 39, 42, 46, 51, 54, 62, 63, 65, 68, 73, 75, 76, 77, 79, 80, 83, 84, 85, 92, 93, 94, 95, 96, 97, 98, 103, 104, 105, 114, 115, 117, 121, 124, 126, 129, 130, 146, 149, 150, 153, 157, 160, 162, 165, 168, 170, 174, 176, 177, 178, 189, 191, 193, 194, 195, 196, 197, 198, 199, 200, 202, 203, 205, 207, 213, 214, 219, 220, 221, 222, 225, 226, 228, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 243, 244, 245, 250, 257, 259, 260, 263, 265, 266, 269, 272, 275, 276, 277, 279, 284, 285, 286, 297, 301, 302, 303, 304, 306, 310, 311, 312, 314, 315, 316, 317, 320, 322, 323, 324, 328, 329, 330, 332, 333, 334, 335, 336, 337, 341, 343, 344, 345, 346, 347, 348, 351, 354, 355, 357, 359, 362, 363, 364, 365, 367, 368, 369, 370, 373, 374, 375, 376, 377, 378, 379, 385, 393, 394, 395, 399, 400, 404, 405, 411, 412, 420, 422, 423, 424, 427, 428, 430, 439, 440, 450, 451, 452, 453, 457, 458, 459, 462, 464, 465, 469, 473, 474, 476, 477, 478, 481, 484, 487, 488, 492, 493, 494, 495, 496,

Então pra mais da metade das leis Count e TF-IDF retornam resultados diferentes. Será que de TF-IDF pra vetor de palavras vai ter essa diferença toda também? Uma questão. 

Voltando ao que interessa, vamo dar uma olhada em mais algumas leis pra continuar a comparação entre Count e TF-IDF.

Vamo sortear indices aleatorios desse vetor e ler as leis que eles representam e as similaridades

In [35]:
num_leis_sorteadas = 10
idx_dif = np.random.randint(0, high=len(difs)-1, size=num_leis_sorteadas)
sorteadas = [difs[i] for i in idx_dif]
print(sorteadas)

[1018, 5776, 2789, 1772, 530, 4810, 5238, 4660, 5383, 4699]


In [39]:
# Exibir as 10 leis:
for i in sorteadas:
    print(f'\n\nLei numero {i}\nPAR UTILIZANDO COUNT:\n\n')
    print_lei_mais_sem(i, max_sim_index[i])
    print('\n\nPAR UTILIZANDO TF-IDF:\n\n')
    print_lei_mais_sem(i, max_sim_idx_tfidf[i])



Lei numero 1018
PAR UTILIZANDO COUNT:


- - - LEI COMPARADA: - - -


faco saber camara atraves edil roberto luis silva tourinho proibida ambito administracao publica concessao homenagens pessoas sido condenadas sentencas transitados julgado improbidade administracao publica crime corrupcao paragrafo unico incluem vedacao caput denominacao predios locais publicos municipais vedacao dispoe estende tambem pessoas sido condenadas sentencas transitadas julgado pratica crimes contra direitos humanos exploracao trabalho escravo maus tratos animais sido historicamente considerados participantes condenados dolosos contra vida entrara publicacao revogadas disposicoes contrario setembro jose ronaldo carvalho mario costa borges chefe cleudson santos almeida procurador justiniano oliveira franca secretario servicos publicos publicado diario oficial eletronico dia outubro


- - - LEI MAIS SEMELHANTE - - -


faco saber camara atraves edil roberto luis silva tourinho considerada utilidade publica as

### Lei 1018
Lei 1018 é sobre proibição de homenagens a condenados por corrupção. Count trouxe uma lei sobre tornar uma associação pública. TF-IDF trouxe uma lei sobre evento de comemoração de adoção animal. Todas duas erraram. TF-IDF chegou mais perto? Difícil dizer

### Lei 5776
Lei 5776 sobre pagamento servidor público. Count trouxe: leitura da bíblia na abertura da câmara. TF-IDF: aposentadoria diretor valor vencimento etc. Ambas as leis parecem ter sido trazidas como semelhantes pq dos nomes próprios contidos nas leis.

### Lei 2789
Lei 2789 (mil anos da revolução francesa) sobre obrigatoriedade de um servidor formado em primeiros socorros em escolas. Count: faço saber inkaba instituto de karate. TF-IDF: faço saber associação estrela jaco. Novamente as semelhanças são os nomes próprios nas leis.

### Lei 1772
Lei 1772: faço saber sindicato trabalhadores rurais. Count: faço saber associação profissionais sexo. TF-IDF: faço saber associação pequenos agricultures apaeb. A rua da sede é a mesma da lei comparada. 

TF-IDF se saiu melhor nessa. Os nomes das pessoas em Count eram os mesmos da Lei, mas em TF-IDF não. O fator decisivo aqui foi o nome da rua, que era o mesmo. Ponto pra TF-IDF.

### Lei 503
Lei 530: faço saber igreja ministerio pentecostal fogo gloria. rua volta redonda bairro campo limpo.
Count: faço saber instituto nobre sede rodovia br km cis. nomes das pessoas iguais.
TF-IDF: faço saber igreja evangelica pentecostal monte carmelo rua espassonavel bairro george americo. prefeitos diferentes.

### Lei 4810
Lei: comenda. nomes: godofredo rebell figueiredo filho, raymundo luiz oliveira lopes. 
Count: comenda. nomes: godofredo rebello figueiredo filho, nilton bellas vieira.
TF-IDF: comenda. nomes: godofredo rebello figueiredo filho, raimundo antonio carneiro pinto.

As duas acertaram. TF-iDF nome mais parecido? Não deve influenciar...

### Lei 5238 
Lei 5238: promulgação de novas vias públicas. A via por TF-IDF passa por mais ruas semelhantes.

### Lei 5383
Lei promulga academia de ginástica. Count: promulga empresas serviço funerario. TF-IDF: promulga novos aparelhos de ginástica.

### Outras
As outras leis eram: _visualizar legislativo ba_. Ambas trouxeram textos idênticos.